In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

from datetime import datetime

train_path = './data/train_V2.csv'
test_path = './data/test_V2.csv'

In [2]:
# class get_data:
#     def __init__(self, path, is_train=True):
#         if is_train:
#             self.df = pd.read_csv(path, nrows=500000)
#             self.shuffle_data()
#         else:
#             self.df = pd.read_csv(path)
            
#     def shuffle_data(self):
#         self.df = self.df.reindex(np.random.permutation(self.df.index))
    
#     def fillna(self):
#         self.df['winPlacePerc'] = self.df

class get_data:
    def __init__(self, path, is_train=True):
        self.is_train = is_train
        data = pd.read_csv(path)
        if is_train:
            data = data.sample(100000)
            self.feature = data.drop('winPlacePerc', axis=1)
            self.label = data['winPlacePerc']
        else:
            self.feature = data
        del data
        
        self.deal_feature()
        self.x_train = self.feature
        if is_train:
            self.y_train = self.label
#         if is_train:
#             self.ts_f, self.ts_l, self.vs_f, self.vs_l = self.split_t_v()
        
    def deal_feature(self):
        self.add_cols()
        self.fillna()
        self.drop_cols()
#         self.featuring()
#         self.ohencode()
        self.drop_ids()
            
    def add_cols(self):
        self.feature['teamPlayers'] = self.feature['groupId'].map(self.feature['groupId'].value_counts())
        self.feature['gamePlayers'] = self.feature['matchId'].map(self.feature['matchId'].value_counts())
        self.feature['enemyPlayers'] = self.feature['gamePlayers'] - self.feature['teamPlayers']
        self.feature['totalDistance'] = self.feature['rideDistance'] + self.feature['swimDistance'] + self.feature['walkDistance']
        self.feature['enemyDamage'] = self.feature['assists'] + self.feature['kills']
        
        totalKills = self.feature.groupby(['matchId', 'groupId']).agg({'kills': lambda x: x.sum()})
        totalKills.rename(columns={'kills': 'squadKills'}, inplace=True)
        self.feature = self.feature.join(other=totalKills, on=['matchId', 'groupId'])
        
        self.feature['medicKits'] = self.feature['heals'] + self.feature['boosts']
        self.feature['medicPerKill'] = self.feature['medicKits'] / self.feature['enemyDamage']
        self.feature['distancePerHeals'] = self.feature['totalDistance'] / self.feature['heals']
        self.feature['headShotKillRatio'] = self.feature['headshotKills'] / self.feature['kills']
        self.feature['headshotKillRate'] = self.feature['headshotKills'] / self.feature['kills']
        self.feature['killPlaceOverMaxPlace'] = self.feature['killPlace'] / self.feature['maxPlace']
        self.feature['kills/walkDistance'] = self.feature['kills'] / self.feature['walkDistance']
        self.feature['avgKills'] = self.feature['squadKills'] / self.feature['teamPlayers']
        self.feature['damageRatio'] = self.feature['damageDealt'] / self.feature['enemyDamage']
        self.feature['distTravelledPerGame'] = self.feature['totalDistance'] / self.feature['matchDuration']
        self.feature['killPlacePerc'] = self.feature['killPlace'] / self.feature['gamePlayers']
        self.feature['playerSkill'] = self.feature['headshotKills'] + self.feature['roadKills'] + self.feature['assists'] - (5 * self.feature['teamKills'])
        self.feature['gamePlacePerc'] = self.feature['killPlace'] / self.feature['maxPlace']
    
    def fillna(self):
        self.feature.fillna(0, inplace=True)
        self.feature.replace(np.inf, 0, inplace=True)
        if self.is_train:
            self.label.fillna(0, inplace=True)
            self.label.replace(np.inf, 0, inplace=True)
            
#     def fillInf(self, val):
#         numcols = self.feature.select_dtypes(include='number').columns
#         cols = numcols[numcols != 'winPlacePerc']
#         self.feature[self.feature == np.Inf] = np.NaN
#         self.feature[self.feature == np.NINF] = np.NaN
#         for c in cols:
#             self.feature[c].fillna(val, inplace=True)
        
    def ohencode(self):
        '''
        solo  <-- solo,solo-fpp,normal-solo,normal-solo-fpp
        duo   <-- duo,duo-fpp,normal-duo,normal-duo-fpp,crashfpp,crashtpp
        squad <-- squad,squad-fpp,normal-squad,normal-squad-fpp,flarefpp,flaretpp
        '''
        mapper = lambda x: 'solo' if ('solo' in x) else 'duo' if ('duo' in x) or ('crash' in x) else 'squad'
        self.feature['matchType'] = self.feature['matchType'].apply(mapper)

        self.feature = pd.concat([self.feature, pd.get_dummies(self.feature['matchType'], prefix='matchType')], axis=1)
    
    def drop_cols(self):
        drop_cols = ['killPoints', 'rankPoints', 'winPoints', 'maxPlace']
        self.feature.drop(columns=drop_cols, inplace=True)
    
    def drop_ids(self):
        self.feature = self.feature.drop(['Id', 'groupId', 'matchId', 'matchType'], axis=1)
    
    def featuring(self):
        features = list(self.feature.columns)
        features.remove("Id")
        features.remove("matchId")
        features.remove("groupId")
        features.remove("matchType")
        condition='False'
        
        if 'winPlacePerc' in self.feature.columns:
            y = np.array(self.feature.groupby(['matchId','groupId'])['winPlacePerc'].agg('mean'), dtype=np.float64)
            features.remove("winPlacePerc")
            condition='True'
        
        # get group mean feature
        agg = self.feature.groupby(['matchId','groupId'])[features].agg('mean')
        agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
        df_out = agg.reset_index()[['matchId','groupId']]
        df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
        df_out = df_out.merge(agg_rank, suffixes=["_mean", "_mean_rank"], how='left', on=['matchId', 'groupId'])
    
        # get group max feature
        agg = self.feature.groupby(['matchId','groupId'])[features].agg('max')
        agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
        df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
        df_out = df_out.merge(agg_rank, suffixes=["_max", "_max_rank"], how='left', on=['matchId', 'groupId'])
        
        # get group min feature
        agg = self.feature.groupby(['matchId','groupId'])[features].agg('min')
        agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
        df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
        df_out = df_out.merge(agg_rank, suffixes=["_min", "_min_rank"], how='left', on=['matchId', 'groupId'])
        
        # get match mean feature
        agg = self.feature.groupby(['matchId'])[features].agg('mean').reset_index()
        df_out = df_out.merge(agg, suffixes=["", "_match_mean"], how='left', on=['matchId'])
        df_id=df_out[["matchId", "groupId"]].copy()
        df_out.drop(["matchId", "groupId"], axis=1, inplace=True)

        del agg, agg_rank
        gc.collect()
        if condition == 'True':
            return df_out,pd.DataFrame(y),df_id
        else:
            return df_out,df_id
        
    def split_t_v(self):
        ts_f, vs_f, ts_l, vs_l = \
        train_test_split(
            self.feature,
            self.label,
            test_size=0.2,
            random_state=42
        )
        return ts_f, ts_l, vs_f, vs_l

In [4]:
class create_model:
    def __init__(
        self,
        periods = 10,
        steps = 10000,
        save_model = True,
        save_model_name = 'test'
    ):
        self.periods = periods
        self.steps = steps
        
        self.save_model = save_model
        self.save_model_path = './model/estimator' + save_model_name
        
        self.create_feature_columns()
        self.create_optimizer()
        
        model_arg = dict(
            feature_columns = self.featureCols,
            hidden_units = [1024, 128, 32],
            optimizer = self.optimizer,
            model_dir = name
        )
        if self.save_model:
            model_arg['model_dir'] = self.save_model_path
        self.model = tf.estimator.DNNRegressor(**model_arg)
    
    def create_feature_columns(self):
        featureColumns = []
        featureColumns.append(tf.feature_column.numeric_column('boosts'))
        featureColumns.append(tf.feature_column.numeric_column('headshotKills'))
        featureColumns.append(tf.feature_column.numeric_column('heals'))
        featureColumns.append(tf.feature_column.numeric_column('kills'))
        featureColumns.append(tf.feature_column.numeric_column('walkDistance'))
        featureColumns.append(tf.feature_column.numeric_column('weaponsAcquired'))
        self.featureCols = featureColumns
        
    def create_optimizer(self):
        self.optimizer = tf.train.ProximalAdagradOptimizer(
            learning_rate = 0.003,
            l1_regularization_strength = 0.001
        )

    def train(
        self,
        ts_f,
        ts_l,
        vs_f,
        vs_l
    ):
        self.ts_f = ts_f
        self.ts_l = ts_l
        self.vs_f = vs_f
        self.vs_l = vs_l
        self.create_input_fn()
        
        print('training start.')
        step_period = self.steps / self.periods
        for period in range(self.periods):
            evaluate = self.model.evaluate(input_fn=self.eval_fn, steps=step_period)
            training = self.model.train(input_fn=self.train_fn, steps=step_period)
            print('period: ', period)
        evaluate = self.model.evaluate(input_fn=self.eval_fn, steps=step_period)
        print('training done.')
            
    def create_input_fn(self):
        self.create_train_fn()
        self.create_eval_fn()
        
    def create_train_fn(self):
        self.train_fn = tf.estimator.inputs.pandas_input_fn(
            x = self.ts_f,
            y = self.ts_l,
            batch_size = 30,
            num_epochs = None,
            shuffle = True,
        )
        
    def create_eval_fn(self):
        self.eval_fn = tf.estimator.inputs.pandas_input_fn(
            x = self.vs_f,
            y = self.vs_l,
            batch_size = 1,
            num_epochs = 1,
            shuffle = True,
        )

    def predict(self, test_df):
        self.test_df = test_df
        self.create_predict_fn()
        prediction = self.model.predict(input_fn=self.predict_fn)
        prediction = [item['predictions'][0] for item in prediction]
        
        self.result = pd.DataFrame(
            [(Id, pred) for Id, pred in zip(testData['Id'], prediction)],
            columns = ['Id', 'winPlacePerc']
        )
        
    def create_predict_fn(self):
        self.predict_fn = tf.estimator.inputs.pandas_input_fn(
            x = self.test_df,
            y = None,
            batch_size = 1,
            num_epochs = 1,
            shuffle = False,
        )
    
    def save_result(self, path='./result/estimator/submission.csv'):
        self.result.to_csv(path, index=False)

In [6]:
train = get_data(train_path)
train.feature.head()

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,kills,killStreaks,longestKill,...,headShotKillRatio,headshotKillRate,killPlaceOverMaxPlace,kills/walkDistance,avgKills,damageRatio,distTravelledPerGame,killPlacePerc,playerSkill,gamePlacePerc
0,0,0,0.00,0,0,0,60,0,0,0.00,...,0.0,0.0,2.142857,0.000000,1.0,0.0,0.187443,0.625000,0,2.142857
1,0,0,91.47,0,0,0,57,0,0,0.00,...,0.0,0.0,2.192308,0.000000,1.0,0.0,0.813193,0.626374,0,2.192308
2,1,0,68.00,0,0,0,47,0,0,0.00,...,0.0,0.0,0.940000,0.000000,1.0,68.0,0.122762,0.479592,1,0.940000
3,0,0,32.90,0,0,0,75,0,0,0.00,...,0.0,0.0,2.419355,0.000000,0.0,0.0,0.141156,0.824176,0,2.419355
4,0,0,100.00,0,0,0,45,1,1,58.53,...,0.0,0.0,0.463918,0.020101,1.0,100.0,0.034937,0.463918,0,0.463918
